In [23]:
import importlib
import model.trainer
import torch
import data.VCTK
importlib.reload(model.trainer)
importlib.reload(data.VCTK)

<module 'data.VCTK' from '/home/azureuser/dom/Project/data/VCTK.py'>

In [24]:
trainer = model.trainer.Trainer(
    device=torch.device("cuda"),
    checkpoint=7,
    load_from_checkpoint=True
)

Using default model
Using default vocoder
Loading models from checkpoint 7
Models loaded


In [25]:
dataset = model.trainer.get_dataset()

In [27]:
i=0
while True:
    total_loss, spectro_loss, vocoder_loss, (spectro_pred, wav_pred, wav_pred_temp) = trainer.train_step(dataset[0])
    i+=1
    vocoder_loss = vocoder_loss.detach().to("cpu").numpy()
    print(i, vocoder_loss)
    if vocoder_loss < 0:
        break

In [ ]:
print(trainer.vocoder_loss_func(dataset[0][1], wav_pred.to("cpu")))
print(trainer.vocoder_loss_func(wav_pred.to("cpu"), dataset[0][1]))

tensor(27466.0430, grad_fn=<DivBackward1>)
tensor(-1.1806, grad_fn=<DivBackward1>)


In [ ]:
wav_pred_temp.shape

torch.Size([2, 25600])